In [1]:
import xarray as xr
import numpy as np
import zarr
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os, sys, time, glob, re
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.feature import NaturalEarthFeature, LAND, COASTLINE

from herbie import Herbie

# set the directory to current directory
root_dir = '/data/harish'
os.chdir(root_dir)

In [2]:
zarr_path = f'{root_dir}/rtma_i10fg_NYS_subset.zarr'
ds = xr.open_zarr(zarr_path)
ds

<xarray.Dataset>
Dimensions:            (time: 52453, y: 256, x: 384)
Coordinates:
    heightAboveGround  float64 ...
    latitude           (y, x) float64 dask.array<chunksize=(256, 384), meta=np.ndarray>
    longitude          (y, x) float64 dask.array<chunksize=(256, 384), meta=np.ndarray>
    step               timedelta64[ns] ...
  * time               (time) datetime64[ns] 2018-01-01 ... 2023-12-31T23:00:00
    valid_time         (time) datetime64[ns] dask.array<chunksize=(26227,), meta=np.ndarray>
Dimensions without coordinates: y, x
Data variables:
    i10fg              (time, y, x) float32 dask.array<chunksize=(1, 256, 384), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_edition:            2
    GRIB_subCentre:          4
    history:                 2024-09-16T16:46 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             US National Weather Service - NCEP

In [3]:
def map_plotter(fig,gs,data,x,y,levels,cmap,title,shrink,colorbar,cbar_label,orientation,extent=None,rectangles=None,sample_points=None,fontsize=None,pad=None):    
    ax = fig.add_subplot(gs,projection=ccrs.PlateCarree())
    ax.coastlines()
    
    contour = data.plot.contourf(
        x=x,y=y,add_colorbar=False,cmap = cmap,extend='both',levels=levels,
        #cbar_kwargs = {'orientation':orientation, 'shrink':shrink, 'aspect':40, 'label':cbar_label,'fontsize':fontsize},
        ax=ax)
    
    # Add colorbar with font size
    if colorbar:
        cbar = fig.colorbar(contour, orientation=orientation, shrink=shrink, aspect=40, label=cbar_label,pad=pad)
        cbar.ax.tick_params(labelsize=fontsize)
        if orientation == 'horizontal':
            cbar.ax.set_xlabel(cbar_label, fontsize=fontsize)
            cbar.ax.xaxis.set_label_position('top')
        else:
            cbar.ax.set_ylabel(cbar_label, fontsize=fontsize)
            cbar.ax.yaxis.set_label_position('left')
            
    
    if extent:
        # Set the extent (for example, bounding box for Europe)
        ax.set_extent(extent, crs=ccrs.PlateCarree())
    
    # Add gridlines with labels
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    
    # Set x and y ticks font size
    gl.xlabel_style = {'size': fontsize}
    gl.ylabel_style = {'size': fontsize}
    
    # set title
    ax.set_title(title,fontsize=fontsize)
    
    # Add land feature with grey color
    land_feature = cfeature.NaturalEarthFeature(
        'physical', 'land', '10m', edgecolor='face', facecolor='lightgrey'
    )
    #ax.add_feature(land_feature)
    ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=1, edgecolor='black')
    
    # Add country borders
    ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=1, edgecolor='black')
    
    # Add U.S. state boundaries
    ax.add_feature(cfeature.STATES, linestyle='-', linewidth=0.5, edgecolor='black')  # U.S. state boundaries
    
    if sample_points:
        for label, (lat, lon) in sample_points.items():
            ax.text(lon, lat - 0.05, label, color='black', fontsize=fontsize, ha='center', va='center')
            ax.plot(lon, lat, '.', markersize=8,color='black')
    
    if rectangles:
        target_grid = {'min_lat': 41.25, 'min_lon': -9.35,'max_lat':41.8,'max_lon':-8.65}
        # Draw a rectangular boxes
        min_lon = target_grid['min_lon']
        min_lat = target_grid['min_lat']
        max_lon = target_grid['max_lon']
        max_lat = target_grid['max_lat']
        rect = matplotlib.patches.Rectangle((min_lon, min_lat), max_lon - min_lon, max_lat - min_lat,
                                        linestyle='--', linewidth=1, edgecolor='red', facecolor='none',zorder=10)
        ax.add_patch(rect)
        
    return ax

# Event: Tornado damaege in Lewis County
- Date: August 5-8, 2023

In [ ]:
event_name = 'Tornado_Lewis'
# create a directory with the event_name even if exist
event_dir = f'/home/harish/OneDrive_UAlbany/gUstNET/Events/{event_name}'
os.makedirs(event_dir, exist_ok=True)

# create time slices in the format of '2023-08-05T00' from '2023-08-05T00:00:00' to '2023-08-08T23:00:00'
time_slices = pd.date_range(start='2023-08-05T00:00:00', end='2023-08-08T23:00:00', freq='h').strftime('%Y-%m-%dT%H')
for i,time_slice in enumerate(time_slices):
    fig = plt.figure(figsize=(8,7), dpi=300)
    gs = matplotlib.gridspec.GridSpec(1, 1)
    map_plotter(fig,gs[0,0],ds.i10fg.sel(time=time_slice),x='longitude',y='latitude',levels=np.arange(0,25.1,1),cmap='plasma',
                    title=f'Instantaneous GUST, {time_slice}',shrink=1,colorbar=True,cbar_label='m/s',
                    orientation='horizontal',fontsize=8,pad=0.1)
    plt.savefig(f'{event_dir}/{time_slice}.png',bbox_inches='tight')
    plt.close(fig)

# Event: Heavy wind ind eastern/northern NY
- Date: March 24-26, 2023

In [7]:
event_name = 'Heavi_wind_east_north_NY'
# create a directory with the event_name even if exist
event_dir = f'/home/harish/OneDrive_UAlbany/gUstNET/Events/{event_name}'
os.makedirs(event_dir, exist_ok=True)

# create time slices in the format of '2023-08-05T00' from '2023-08-05T00:00:00' to '2023-08-08T23:00:00'
time_slices = pd.date_range(start='2023-03-24T00:00:00', end='2023-03-26T23:00:00', freq='h').strftime('%Y-%m-%dT%H')
for i,time_slice in enumerate(time_slices):
    fig = plt.figure(figsize=(8,7), dpi=300)
    gs = matplotlib.gridspec.GridSpec(1, 1)
    map_plotter(fig,gs[0,0],ds.i10fg.sel(time=time_slice),x='longitude',y='latitude',levels=np.arange(0,25.1,1),cmap='plasma',
                    title=f'Instantaneous GUST, {time_slice}',shrink=1,colorbar=True,cbar_label='m/s',
                    orientation='horizontal',fontsize=8,pad=0.1)
    plt.savefig(f'{event_dir}/{time_slice}.png',bbox_inches='tight')
    plt.close(fig)

# Event: Tornado and heavy wind in Ulster, Washington, Onondaga, and Suffolk counties
- Date: July 22-24, 2022

In [8]:
event_name = 'Tornado_heavy_wind_Ulster_Washington_Onondaga_Suffolk'
# create a directory with the event_name even if exist
event_dir = f'/home/harish/OneDrive_UAlbany/gUstNET/Events/{event_name}'
os.makedirs(event_dir, exist_ok=True)

# create time slices in the format of '2023-08-05T00' from '2023-08-05T00:00:00' to '2023-08-08T23:00:00'
time_slices = pd.date_range(start='2022-07-22T00:00:00', end='2022-07-24T23:00:00', freq='h').strftime('%Y-%m-%dT%H')
for i,time_slice in enumerate(time_slices):
    fig = plt.figure(figsize=(8,7), dpi=300)
    gs = matplotlib.gridspec.GridSpec(1, 1)
    map_plotter(fig,gs[0,0],ds.i10fg.sel(time=time_slice),x='longitude',y='latitude',levels=np.arange(0,25.1,1),cmap='plasma',
                    title=f'Instantaneous GUST, {time_slice}',shrink=1,colorbar=True,cbar_label='m/s',
                    orientation='horizontal',fontsize=8,pad=0.1)
    plt.savefig(f'{event_dir}/{time_slice}.png',bbox_inches='tight')
    plt.close(fig)

# Event: Heavy wind in western and northern NY
- Date: December 10, 2021

In [9]:
event_name = 'Heavy_wind_western_northern_NY'
# create a directory with the event_name even if exist
event_dir = f'/home/harish/OneDrive_UAlbany/gUstNET/Events/{event_name}'
os.makedirs(event_dir, exist_ok=True)

# create time slices in the format of '2023-08-05T00' from '2023-08-05T00:00:00' to '2023-08-08T23:00:00'
time_slices = pd.date_range(start='2021-12-10T00:00:00', end='2021-12-10T23:00:00', freq='h').strftime('%Y-%m-%dT%H')
for i,time_slice in enumerate(time_slices):
    fig = plt.figure(figsize=(8,7), dpi=300)
    gs = matplotlib.gridspec.GridSpec(1, 1)
    map_plotter(fig,gs[0,0],ds.i10fg.sel(time=time_slice),x='longitude',y='latitude',levels=np.arange(0,25.1,1),cmap='plasma',
                    title=f'Instantaneous GUST, {time_slice}',shrink=1,colorbar=True,cbar_label='m/s',
                    orientation='horizontal',fontsize=8,pad=0.1)
    plt.savefig(f'{event_dir}/{time_slice}.png',bbox_inches='tight')
    plt.close(fig)

# Event: Severe storm in SE NY
- Date: February 23-25, 2019

In [10]:
event_name = 'Severe_storm_SE_NY'
# create a directory with the event_name even if exist
event_dir = f'/home/harish/OneDrive_UAlbany/gUstNET/Events/{event_name}'
os.makedirs(event_dir, exist_ok=True)

# create time slices in the format of '2023-08-05T00' from '2023-08-05T00:00:00' to '2023-08-08T23:00:00'
time_slices = pd.date_range(start='2019-02-23T00:00:00', end='2019-02-25T23:00:00', freq='h').strftime('%Y-%m-%dT%H')
for i,time_slice in enumerate(time_slices):
    fig = plt.figure(figsize=(8,7), dpi=300)
    gs = matplotlib.gridspec.GridSpec(1, 1)
    map_plotter(fig,gs[0,0],ds.i10fg.sel(time=time_slice),x='longitude',y='latitude',levels=np.arange(0,25.1,1),cmap='plasma',
                    title=f'Instantaneous GUST, {time_slice}',shrink=1,colorbar=True,cbar_label='m/s',
                    orientation='horizontal',fontsize=8,pad=0.1)
    plt.savefig(f'{event_dir}/{time_slice}.png',bbox_inches='tight')
    plt.close(fig)